# Imports

In [1]:
import numpy as np
import os
from os import listdir as LD, makedirs as MDs
from os.path import join as PJ, basename as PBN, dirname as PDN, exists as PE
import pandas as pd
from datetime import datetime as DT

In [2]:
from imod import msw
from imod import mf6
import primod

In [3]:
import WS_Mdl.utils as U
import WS_Mdl.utils_imod as UIM

In [39]:
import importlib as IL
IL.reload(U)
IL.reload(UIM)

<module 'WS_Mdl.utils_imod' from 'C:\\Users\\Karam014\\OneDrive - Universiteit Utrecht\\WS_Mdl\\code\\WS_Mdl\\utils_imod.py'>

# Options + Basics

In [5]:
MdlN = 'NBr32'

In [6]:
U.set_verbose(False)

In [7]:
# Load paths and variables from PRJ & INI
d_Pa = U.get_MdlN_Pa(MdlN)
Pa_PRJ = d_Pa['PRJ']
Dir_PRJ = PDN(Pa_PRJ)
d_INI = U.INI_to_d(d_Pa['INI'])
Xmin, Ymin, Xmax, Ymax = [float(i) for i in d_INI['WINDOW'].split(',')]
SP_date_1st, SP_date_last = [DT.strftime(DT.strptime(d_INI[f'{i}'], '%Y%m%d'), '%Y-%m-%d') for i in ['SDATE', 'EDATE']]
dx = dy = float(d_INI['CELLSIZE'])

# Load PRJ

In [8]:
PRJ_, PRJ_OBS = UIM.open_PRJ_with_OBS(Pa_PRJ)

In [48]:
PRJ, period_data = PRJ_[0], PRJ_[1]

In [49]:
# PRJ['msw'] = PRJ.pop('cap') # Rename 'cap' to 'msw' for consistency

# Load DIS and limit to Mdl Aa

In [50]:
U.set_verbose(True)
PRJ_regrid = UIM.regrid_PRJ(PRJ, MdlN)

🟢 - INI file C:/OD/WS_Mdl\models/NBr\code/Mdl_Prep/Mdl_Prep_NBr32.ini read successfully. Dictionary created with 21 keys.
🟢 - model dimensions extracted from C:/OD/WS_Mdl\models/NBr\code/Mdl_Prep/Mdl_Prep_NBr32.ini.

Target grid: 480x344 cells at 25.0x-25.0 m resolution

Target extents: X=[113112.5, 125087.5], Y=[387612.5, 396187.5]
Processing cap...
  cap.soil_physical_unit: 🟢 - Frozen({'layer': 1, 'y': 4504, 'x': 6012}) -> Frozen({'layer': 1, 'y': 344, 'x': 480})
  cap.urban_infiltration_capacity: ⚪️ - No spatial dims - keeping original
  cap.urban_runoff_resistance: ⚪️ - No spatial dims - keeping original
  cap.meteo_station_number: 🟢 - Frozen({'layer': 1, 'y': 1126, 'x': 1503}) -> Frozen({'layer': 1, 'y': 344, 'x': 480})
  cap.urban_runon_resistance: ⚪️ - No spatial dims - keeping original
  cap.rural_infiltration_capacity: ⚪️ - No spatial dims - keeping original
  cap.artificial_recharge: 🟢 - Frozen({'layer': 1, 'y': 4504, 'x': 6012}) -> Frozen({'layer': 1, 'y': 344, 'x': 480})
  

# Load MF6 Mdl

In [51]:
times = pd.date_range(SP_date_1st, SP_date_last, freq='D')

In [52]:
# PRJ_no_MSW = PRJ.copy()
# PRJ_MSW = {}
# if 'msw' in PRJ_no_MSW:
#     PRJ_MSW['msw'] = PRJ_no_MSW['msw']
#     PRJ_MSW['extra'] = PRJ_no_MSW['extra']
#     del PRJ_no_MSW['msw'], PRJ_no_MSW['extra']
#     print("Removed MSW/CAP package due to mixed grid compatibility issues")

# Sim_MF6 = mf6.Modflow6Simulation.from_imod5_data(PRJ_no_MSW, period_data, times)

In [53]:
# # Convert IMOD5 to MODFLOW6 (with proper error handling)
# # First separate MSW package and fix temporal data issues

# print("=== PREPARING PRJ DATA FOR MF6 CONVERSION ===")

# PRJ_no_MSW = PRJ_regrid.copy()
# PRJ_MSW = {}
# if 'msw' in PRJ_no_MSW:
#     PRJ_MSW['msw'] = PRJ_no_MSW['msw']
#     PRJ_MSW['extra'] = PRJ_no_MSW['extra']
#     del PRJ_no_MSW['msw'], PRJ_no_MSW['extra']
#     print("Removed MSW/CAP package due to mixed grid compatibility issues")

# # Check for problematic temporal data in river packages
# riv_keys = [key for key in PRJ_no_MSW.keys() if key.startswith('riv')]
# print(f"Found river packages: {riv_keys}")

# for riv_key in riv_keys:
#     if hasattr(PRJ_no_MSW[riv_key], 'time') and 'repeat_stress' in PRJ_no_MSW[riv_key].coords:
#         repeat_stress = PRJ_no_MSW[riv_key].coords['repeat_stress']
#         print(f"{riv_key} repeat_stress dtype: {repeat_stress.dtype}")
        
#         # Fix mixed data types in repeat_stress if needed
#         if repeat_stress.dtype != 'datetime64[ns]':
#             print(f"⚠️  Fixing {riv_key} repeat_stress data type...")
#             try:
#                 if np.issubdtype(repeat_stress.dtype, np.number):
#                     # Convert numeric to datetime
#                     PRJ_no_MSW[riv_key] = PRJ_no_MSW[riv_key].assign_coords(
#                         repeat_stress=times[:len(repeat_stress)]
#                     )
#             except Exception as e:
#                 print(f"Could not fix {riv_key}: {e}")
#                 PRJ_no_MSW[riv_key] = PRJ_no_MSW[riv_key].drop_vars('repeat_stress', errors='ignore')

# print("\n=== ATTEMPTING MF6 CONVERSION ===")
# try:
#     Sim_MF6 = mf6.Modflow6Simulation.from_imod5_data(PRJ_no_MSW, period_data, times)
#     print("✅ Simulation created successfully!")
# except Exception as e:
#     print(f"❌ Conversion failed: {e}")
#     print("\nTrying alternative approach...")
    
#     # Alternative: Remove problematic packages temporarily
#     PRJ_minimal = {k: v for k, v in PRJ_no_MSW.items() if not k.startswith('riv')}
#     print(f"Trying conversion without river packages: {list(PRJ_minimal.keys())[:10]}...")
    
#     try:
#         Sim_MF6 = mf6.Modflow6Simulation.from_imod5_data(PRJ_minimal, period_data, times)
#         print("✅ Minimal simulation created successfully!")
#         print("Note: River packages were excluded due to temporal data issues")
#     except Exception as e2:
#         print(f"❌ Even minimal conversion failed: {e2}")
#         Sim_MF6 = None

Using original PRJ to load MF6 Mdl gives warnings (and it's very slow). Thus, well use the regridded PRJ, which is much faster. It can be further sped up by multi-processing, but this is not implemented yet.

In [54]:
PRJ_no_MSW = PRJ_regrid.copy()
PRJ_MSW = {}
if 'cap' in PRJ_no_MSW:
    PRJ_MSW['cap'] = PRJ_no_MSW['cap']
    PRJ_MSW['extra'] = PRJ_no_MSW['extra']
    del PRJ_no_MSW['cap'], PRJ_no_MSW['extra']
    print("Removed cap/extra package due to mixed grid compatibility issues")

Sim_MF6 = mf6.Modflow6Simulation.from_imod5_data(PRJ_no_MSW, period_data, times)

Removed cap/extra package due to mixed grid compatibility issues


In [55]:
MF6_Mdl = Sim_MF6['imported_model']

For some reason there is only 1 river system remaining. I'd expect at least 2 out of 5 (RIV and detailwatergangen) (RIV supposedly has 2, but the one that can discharge into the aquifer has a conductance of 0).

In [56]:
MF6_Mdl["oc"] = mf6.OutputControl(save_head="last", save_budget="last")
    
# Mimic iMOD5's "Moderate" settings
Sim_MF6["ims"] = UIM.mf6_solution_moderate_settings()

# Load MSW

## 1st attempt

In [ ]:
# Don't extract MF6_DIS here - do it after regridding!
# MF6_DIS = Sim_MF6["imported_model"]["dis"]  # This gets the OLD 100m grid
print("MF6_DIS will be extracted after regridding to ensure 25m resolution")

In [58]:
PRJ_regrid.keys()

dict_keys(['cap', 'extra', 'bnd', 'top', 'bot', 'khv', 'kva', 'shd', 'sto', 'spy', 'hfb-1', 'hfb-2', 'hfb-3', 'hfb-4', 'hfb-5', 'hfb-6', 'hfb-7', 'hfb-8', 'hfb-9', 'hfb-10', 'hfb-11', 'hfb-12', 'hfb-13', 'hfb-14', 'hfb-15', 'hfb-16', 'hfb-17', 'hfb-18', 'hfb-19', 'hfb-20', 'hfb-21', 'hfb-22', 'hfb-23', 'hfb-24', 'hfb-25', 'hfb-26', 'hfb-27', 'hfb-28', 'hfb-29', 'hfb-30', 'hfb-31', 'hfb-32', 'hfb-33', 'hfb-34', 'hfb-35', 'hfb-36', 'hfb-37', 'hfb-38', 'hfb-39', 'hfb-40', 'hfb-41', 'hfb-42', 'hfb-43', 'hfb-44', 'hfb-45', 'hfb-46', 'hfb-47', 'hfb-48', 'hfb-49', 'hfb-50', 'hfb-51', 'hfb-52', 'hfb-53', 'hfb-54', 'hfb-55', 'hfb-56', 'hfb-57', 'hfb-58', 'hfb-59', 'hfb-60', 'hfb-61', 'hfb-62', 'hfb-63', 'hfb-64', 'hfb-65', 'hfb-66', 'hfb-67', 'hfb-68', 'hfb-69', 'hfb-70', 'hfb-71', 'hfb-72', 'hfb-73', 'hfb-74', 'hfb-75', 'hfb-76', 'hfb-77', 'hfb-78', 'hfb-79', 'hfb-80', 'hfb-81', 'hfb-82', 'hfb-83', 'hfb-84', 'hfb-85', 'hfb-86', 'hfb-87', 'hfb-88', 'hfb-89', 'hfb-90', 'hfb-91', 'hfb-92', 'hfb-9

In [60]:
MF6_DIS['x']

<xarray.DataArray 'x' (x: 121)> Size: 968B
array([113112.5, 113212.5, 113312.5, 113412.5, 113512.5, 113612.5, 113712.5,
       113812.5, 113912.5, 114012.5, 114112.5, 114212.5, 114312.5, 114412.5,
       114512.5, 114612.5, 114712.5, 114812.5, 114912.5, 115012.5, 115112.5,
       115212.5, 115312.5, 115412.5, 115512.5, 115612.5, 115712.5, 115812.5,
       115912.5, 116012.5, 116112.5, 116212.5, 116312.5, 116412.5, 116512.5,
       116612.5, 116712.5, 116812.5, 116912.5, 117012.5, 117112.5, 117212.5,
       117312.5, 117412.5, 117512.5, 117612.5, 117712.5, 117812.5, 117912.5,
       118012.5, 118112.5, 118212.5, 118312.5, 118412.5, 118512.5, 118612.5,
       118712.5, 118812.5, 118912.5, 119012.5, 119112.5, 119212.5, 119312.5,
       119412.5, 119512.5, 119612.5, 119712.5, 119812.5, 119912.5, 120012.5,
       120112.5, 120212.5, 120312.5, 120412.5, 120512.5, 120612.5, 120712.5,
       120812.5, 120912.5, 121012.5, 121112.5, 121212.5, 121312.5, 121412.5,
       121512.5, 121612.5, 121712.5, 121812.5, 121912.5, 122012.5, 122112.5,
       122212.5, 122312.5, 122412.5, 122512.5, 122612.5, 122712.5, 122812.5,
       122912.5, 123012.5, 123112.5, 123212.5, 123312.5, 123412.5, 123512.5,
       123612.5, 123712.5, 123812.5, 123912.5, 124012.5, 124112.5, 124212.5,
       124312.5, 124412.5, 124512.5, 124612.5, 124712.5, 124812.5, 124912.5,
       125012.5, 125112.5])
Coordinates:
    dx       float64 8B 100.0
    dy       float64 8B -100.0
  * x        (x) float64 968B 1.131e+05 1.132e+05 ... 1.25e+05 1.251e+05

In [65]:
# Check MF6_DIS grid details
print(f"MF6_DIS x length: {len(MF6_DIS['x'])}")
print(f"MF6_DIS y length: {len(MF6_DIS['y'])}")
print(f"MF6_DIS dx: {MF6_DIS['x'][1] - MF6_DIS['x'][0]}")
print(f"MF6_DIS dy: {MF6_DIS['y'][1] - MF6_DIS['y'][0]}")
print(f"MF6_DIS x range: {MF6_DIS['x'].min().values} to {MF6_DIS['x'].max().values}")
print(f"MF6_DIS y range: {MF6_DIS['y'].min().values} to {MF6_DIS['y'].max().values}")

MF6_DIS x length: 121
MF6_DIS y length: 87
MF6_DIS dx: <xarray.DataArray 'x' ()> Size: 8B
np.float64(100.0)
Coordinates:
    dx       float64 8B 100.0
    dy       float64 8B -100.0
MF6_DIS dy: <xarray.DataArray 'y' ()> Size: 8B
np.float64(-100.0)
Coordinates:
    dx       float64 8B 100.0
    dy       float64 8B -100.0
MF6_DIS x range: 113112.5 to 125112.5
MF6_DIS y range: 387587.5 to 396187.5


In [35]:
PRJ_regrid['cap']

{'soil_physical_unit': <xarray.DataArray (layer: 1, y: 344, x: 480)> Size: 1MB
 dask.array<transpose, shape=(1, 344, 480), dtype=float64, chunksize=(1, 344, 480), chunktype=numpy.ndarray>
 Coordinates:
     dx       float64 8B 25.0
     dy       float64 8B -25.0
   * layer    (layer) int64 8B 1
   * x        (x) float64 4kB 1.131e+05 1.131e+05 ... 1.251e+05 1.251e+05
   * y        (y) float64 3kB 3.962e+05 3.962e+05 ... 3.876e+05 3.876e+05,
 'urban_infiltration_capacity': <xarray.DataArray (layer: 1)> Size: 8B
 array([0.])
 Coordinates:
   * layer    (layer) int64 8B 1,
 'urban_runoff_resistance': <xarray.DataArray (layer: 1)> Size: 8B
 array([1.])
 Coordinates:
   * layer    (layer) int64 8B 1,
 'urban_area': <xarray.DataArray (layer: 1, y: 344, x: 480)> Size: 1MB
 dask.array<mul, shape=(1, 344, 480), dtype=float64, chunksize=(1, 344, 480), chunktype=numpy.ndarray>
 Coordinates:
     dx       float64 8B 25.0
     dy       float64 8B -25.0
   * layer    (layer) int64 8B 1
   * x       

In [31]:
try:
    MSW_Mdl = msw.MetaSwapModel.from_imod5_data(PRJ_MSW, MF6_DIS, times)
    print("🎉 MetaSwap model created successfully!")
except Exception as e:
    print(f"Error creating MetaSwap model: {e}")
    MSW_Mdl = None

Error creating MetaSwap model: cannot align objects with join='exact' where index/labels/sizes are not equal along these coordinates (dimensions): 'x' ('x',)


## 2nd attempt

### Regriding

In [ ]:
# Debug the grid alignment issue
print(f"MF6_DIS Type: {type(MF6_DIS)}")
print(f"MF6_DIS Keys: {list(MF6_DIS.keys()) if hasattr(MF6_DIS, 'keys') else 'No keys method'}")
# Access the data from MF6_DIS
if hasattr(MF6_DIS, 'dataset'):
    print(f"MF6_DIS keys: {list(MF6_DIS.dataset.keys())}")
    x_MF6 = MF6_DIS.dataset.x.values
    y_MF6 = MF6_DIS.dataset.y.values
else:
    # Try accessing as a dataset directly
    x_MF6 = MF6_DIS.x.values
    y_MF6 = MF6_DIS.y.values

sample_cap = PRJ_MSW['cap']['urban_area']
x_MSW, y_MSW = sample_cap.x.values, sample_cap.y.values

print(f"\nMF6 X range: {x_MF6.min():9.1f} - {x_MF6.max():8.1f} ({f'{len(x_MF6):4} cells'}, dx={MF6_DIS.dataset.dx.values:6.1f}m)")
print(f"MF6 Y range: {y_MF6.min():9.1f} - {y_MF6.max():8.1f} ({f'{len(y_MF6):4} cells'}, dy={abs(MF6_DIS.dataset.dy.values):6.1f}m)")
print(f"MSW X range: {x_MSW.min():9.1f} - {x_MSW.max():8.1f} ({f'{len(x_MSW):4} cells'}, dx={sample_cap.dx.values:6.1f}m)")
print(f"MSW Y range: {y_MSW.min():9.1f} - {y_MSW.max():8.1f} ({f'{len(y_MSW):4} cells'}, dy={abs(sample_cap.dy.values):6.1f}m)")

Dimensions of MF6_DIS and MetaSWAP files do not align. Former is 100x100, latter is 25x25 m. But first we need to get the Xmin and Xmax right.

In [ ]:
PRJ_MSW['cap'].keys()

In [ ]:
# Access the last key in the 'cap' dictionary. The param doesnt matter, we just want to get dx and dy
dx_MSW = float(PRJ_MSW['cap'][ 'boundary' ].dx)
dy_MSW = float(PRJ_MSW['cap'][ 'boundary' ].dy)
dx_MF6 = float(MF6_DIS.dataset['dx'])
dy_MF6 = float(MF6_DIS.dataset['dy'])

In [ ]:
dx_MSW, dy_MSW, dx_MF6, dy_MF6

In [ ]:
x_MSW_CeCes = np.arange( (Xmin + dx_MSW / 2) , Xmax, dx_MSW)
y_MSW_CeCes = np.arange( (Ymax + dy_MSW / 2) , Ymin, dy_MSW)
print(f"MSW grid x: {x_MSW_CeCes.min():.1f} - {x_MSW_CeCes.max():.1f} ({len(x_MSW_CeCes)} cells)")
print(f"MSW grid y: {y_MSW_CeCes.min():.1f} - {y_MSW_CeCes.max():.1f} ({len(y_MSW_CeCes)} cells)")              

x_MF6_CeCes = np.arange( MF6_DIS.dataset['x'].min() , Xmax, dx_MF6)
y_MF6_CeCes = np.arange( MF6_DIS.dataset['y'].max() , Ymin, dy_MF6)
print(f"MF6 grid x: {x_MF6_CeCes.min():.1f} - {x_MF6_CeCes.max():.1f} ({len(x_MF6_CeCes)} cells)")
print(f"MF6 grid y: {y_MF6_CeCes.min():.1f} - {y_MF6_CeCes.max():.1f} ({len(y_MF6_CeCes)} cells)")              

In [ ]:
# Check if all inputs have the same descritization
for i in MF6_Mdl.keys():
    try:
        print(f"{i}:\t{MF6_Mdl[i].dataset.dx.values:10}, {MF6_Mdl[i].dataset.dy.values:10}")
    except:
        print(f"{i}: - -")

In [ ]:
# Create a refined target discretization
MF6_DIS_Rfn = MF6_DIS.dataset.interp(
    x=x_MSW_CeCes,
    y=y_MSW_CeCes,
    method='linear'
)

In [ ]:
# MF6_DIS.dataset['top'].plot.imshow(cmap='terrain')

In [ ]:
# MF6_DIS_Rfn['top'].plot.imshow(cmap='terrain')

Looks like this worked properly.

In [ ]:
MF6_DIS_Rfn['idomain'] = MF6_DIS_Rfn['idomain'].astype(int)

In [ ]:
MF6_DIS_init = MF6_DIS.copy()

In [ ]:
MF6_DIS_Rfn['y']

In [ ]:
MF6_DIS_init['y']

In [ ]:
type(MF6_DIS), type(MF6_DIS_Rfn)

In [ ]:
# Convert back to StructuredDiscretization
MF6_DIS = mf6.StructuredDiscretization(
    idomain=MF6_DIS_Rfn['idomain'],
    top=MF6_DIS_Rfn['top'],
    bottom=MF6_DIS_Rfn['bottom']
 )
print(f"Refined MF6_DIS shape: {MF6_DIS.dataset.sizes}")

In [ ]:
# Now regrid all CAP data to the exact refined coordinates
PRJ_MSW_regridded = {'cap': {}}
for key, data in PRJ_MSW['cap'].items():
    print(f"Processing {key}: dims = {data.dims}")
    
    # Check if data has spatial dimensions (x, y)
    if 'x' in data.dims and 'y' in data.dims:
        if key == 'wetted_area':
            # For area-related fields, we need to preserve the total area
            # The wetted_area represents the actual area, which should remain as cell area
            regridded_data = data.interp(x=x_MSW_CeCes, y=y_MSW_CeCes, method='linear')
            # Set to full cell area (25m x 25m = 625 m²)
            PRJ_MSW_regridded['cap'][key] = regridded_data * 0 + (dx_MSW * dx_MSW)
        else:
            # For other spatial data, interpolate normally
            PRJ_MSW_regridded['cap'][key] = data.interp(
                x=x_MSW_CeCes,
                y=y_MSW_CeCes,
                method='linear'
            )
    else:
        # Keep non-spatial data as is
        PRJ_MSW_regridded['cap'][key] = data
print("CAP data regridding completed.")

In [ ]:
U.set_verbose(False)
UIM.xr_compare_As( UIM.xr_clip_Mdl_Aa(PRJ_MSW['cap']['soil_physical_unit'], MdlN),
                   PRJ_MSW_regridded['cap']['soil_physical_unit'])

Linear interpolation causes A values to differ by a tiny ammount. But, the values are very close to each other, so this is not a problem.

In [ ]:
# Merge with extra metadata
PRJ_MSW_for_MSW = {**PRJ_MSW_regridded, **{'extra': {**PRJ_MSW['extra']}}}

### Fix mete_grid.inp relative paths
MSW.MetaSwapModel.from_imod5_data is struggling with relative paths, so we'll convert them to full paths. #666 caution, if they're already full paths, this may cause an error.

In [ ]:
# Get the original mete_grid.inp file path
Pa_mete_grid = PRJ_MSW['extra']['paths'][2][0]  # 3rd file (index 2) (by design in imod - i.e. the order can't change)
print(f"Original file: {Pa_mete_grid}")

In [ ]:
# Load mete_grid, edit and save it 
Dir_mete_grid = PDN(Pa_mete_grid)
Pa_mete_grid_AbsPa = PJ( PDN(Pa_mete_grid), 'temp', 'mete_grid.inp')
if not PE(PDN(Pa_mete_grid_AbsPa)):
    MDs(PDN(Pa_mete_grid_AbsPa))

DF = pd.read_csv(Pa_mete_grid, header=None, names=['N', 'Y', 'P', 'PET'])
DF.P = DF.P.apply(lambda x: os.path.abspath( PJ(Dir_PRJ, x) ))
DF.PET = DF.PET.apply(lambda x: os.path.abspath( PJ(Dir_PRJ, x) ))  # Fixed: was DF.P instead of DF.PET

In [ ]:
# Write CSV with proper format to avoid imod parsing issues with newlines
# imod doesn't strip newlines from paths, so we need to format carefully
corrected_lines = []
for index, row in DF.iterrows():
    # Add quotes around paths like the original format
    line = f'{row["N"]},{row["Y"]},"{row["P"]}","{row["PET"]}"'
    corrected_lines.append(line)

# Write without newlines in path columns
with open(Pa_mete_grid_AbsPa, 'w') as f:
    for i, line in enumerate(corrected_lines):
        if i == len(corrected_lines) - 1:  # Last line - no newline
            f.write(line)
        else:
            f.write(line + '\n')

print(f"Created corrected mete_grid.inp: {Pa_mete_grid_AbsPa}") 

In [ ]:
# Replace the mete_grid.inp path in the PRJ_MSW_for_MSW dictionary
PRJ_MSW_for_MSW['extra']['paths'][2][0] = Pa_mete_grid_AbsPa

### Finally load MS Sim

In [ ]:
# Create the MetaSwap model
MSW_Mdl = msw.MetaSwapModel.from_imod5_data(PRJ_MSW_for_MSW, MF6_DIS, times)
print("🎉 MetaSwap model created successfully!")

# Explore MSW_Mdl Keys and Plot Parameters

This section systematically goes through all keys in the MetaSWAP model and plots each parameter with clear visual separation.

### Prep

In [ ]:
# First, let's explore the top-level keys of MSW_Mdl
print("="*80)
print("MSW_Mdl Top-Level Keys:")
print("="*80)
for i, key in enumerate(MSW_Mdl.keys(), 1):
    print(f"{i}. {key}")
print("="*80)

In [ ]:
# Let's explore the structure of each key to understand what we can plot
print("="*80)
print("DETAILED STRUCTURE OF EACH MSW_Mdl KEY:")
print("="*80)

for key in MSW_Mdl.keys():
    print(f"\n{'='*20} {key.upper()} {'='*20}")
    try:
        # Check if it has a dataset attribute (most plotting will be from datasets)
        if hasattr(MSW_Mdl[key], 'dataset'):
            print(f"Type: {type(MSW_Mdl[key])}")
            print(f"Dataset variables: {list(MSW_Mdl[key].dataset.keys())}")
            
            # Show dimensions for each variable
            for var in MSW_Mdl[key].dataset.keys():
                dims = MSW_Mdl[key].dataset[var].dims
                shape = MSW_Mdl[key].dataset[var].shape
                print(f"  - {var}: dims={dims}, shape={shape}")
        else:
            print(f"Type: {type(MSW_Mdl[key])}")
            print(f"Content preview: {str(MSW_Mdl[key])[:200]}...")
    except Exception as e:
        print(f"Error exploring {key}: {e}")

print("\n" + "="*80)

In [ ]:
# Let's focus on identifying which keys have spatial data that can be plotted
print("="*80)
print("PLOTTABLE PARAMETERS IN MSW_Mdl:")
print("="*80)

plottable_params = {}

for key in MSW_Mdl.keys():
    print(f"\n{key.upper()}:")
    try:
        if hasattr(MSW_Mdl[key], 'dataset'):
            spatial_vars = []
            for var in MSW_Mdl[key].dataset.keys():
                dims = MSW_Mdl[key].dataset[var].dims
                # Check if variable has spatial dimensions (x, y)
                if 'x' in dims and 'y' in dims:
                    spatial_vars.append(var)
                    print(f"  ✓ {var} - dims: {dims}")
            plottable_params[key] = spatial_vars
        else:
            print(f"  - No dataset attribute (Type: {type(MSW_Mdl[key]).__name__})")
    except Exception as e:
        print(f"  - Error: {e}")

print(f"\n{'='*80}")
print(f"SUMMARY - Keys with plottable parameters:")
for key, vars_list in plottable_params.items():
    if vars_list:
        print(f"  {key}: {len(vars_list)} parameters - {vars_list}")
print("="*80)

---

## 🎨 **PLOTTING ALL MSW_Mdl PARAMETERS**

Below we systematically plot all spatial parameters from each key in the MetaSWAP model. Each section is clearly separated and labeled for easy navigation.

---

In [ ]:
# Utility function for consistent plotting with clear visual separation
import matplotlib.pyplot as plt
import numpy as np

def plot_msw_parameter(data, param_name, key_name, subunit=None, cmap='viridis', figsize=(10, 8)):
    """
    Plot a MetaSWAP parameter with consistent formatting and clear labeling.
    
    Parameters:
    -----------
    data : xarray.DataArray
        The data to plot
    param_name : str
        Name of the parameter
    key_name : str
        Name of the MSW key (grid, infiltration, etc.)
    subunit : int, optional
        Subunit to plot (for parameters with subunit dimension)
    cmap : str
        Colormap to use
    figsize : tuple
        Figure size
    """
    
    print(f"\n{'='*20} {key_name.upper()}: {param_name} {'='*20}")
    
    # Handle subunit selection if needed
    if 'subunit' in data.dims:
        if subunit is None:
            subunit = 0  # Default to first subunit
        plot_data = data.sel(subunit=subunit)
        title = f"{key_name.title()}: {param_name.replace('_', ' ').title()}\n(Subunit {subunit})"
    else:
        plot_data = data
        title = f"{key_name.title()}: {param_name.replace('_', ' ').title()}"
    
    # Create the plot
    fig, ax = plt.subplots(figsize=figsize)
    
    # Plot with imshow
    im = plot_data.plot.imshow(
        ax=ax,
        cmap=cmap,
        add_colorbar=True,
        cbar_kwargs={'shrink': 0.8}
    )
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('X Coordinate', fontsize=12)
    ax.set_ylabel('Y Coordinate', fontsize=12)
    
    # Add statistics
    valid_data = plot_data.values[~np.isnan(plot_data.values)]
    if len(valid_data) > 0:
        stats_text = f"Min: {valid_data.min():.3f} | Max: {valid_data.max():.3f} | Mean: {valid_data.mean():.3f}"
        ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, 
                bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8),
                fontsize=10, verticalalignment='top')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Dimensions: {plot_data.dims}")
    print(f"Shape: {plot_data.shape}")
    print(f"Data type: {plot_data.dtype}")
    print("="*60)

print("✅ Plotting utility function loaded!")

### 🎨 **PLOTTING ALL MSW MODEL PARAMETERS**

Below we systematically plot all spatial parameters from each key in the MetaSWAP model using a grouped, efficient approach.

In [ ]:
# Define plotting configuration for all MSW model keys
msw_plotting_config = {
    'grid': {
        'emoji': '🌍',
        'description': 'Grid parameters including land use, area, soil properties, and surface characteristics',
        'params': ['area', 'landuse', 'rootzone_depth', 'surface_elevation', 'soil_physical_unit', 'active'],
        'subunit_params': ['area', 'landuse', 'rootzone_depth'],
        'colormaps': {
            'landuse': 'tab10',
            'surface_elevation': 'terrain',
            'default': 'viridis'
        }
    },
    'infiltration': {
        'emoji': '💧', 
        'description': 'Infiltration parameters for water capacity, soil resistance, and storage properties',
        'params': ['infiltration_capacity', 'downward_resistance', 'upward_resistance', 'bottom_resistance', 'extra_storage_coefficient'],
        'subunit_params': ['infiltration_capacity', 'downward_resistance', 'upward_resistance'],
        'colormaps': {
            'default': 'viridis',
            'resistance': 'Blues'
        }
    },
    'ponding': {
        'emoji': '🌊',
        'description': 'Ponding parameters for surface water, runoff, and runon processes',
        'params': ['ponding_depth', 'runon_resistance', 'runoff_resistance'],
        'subunit_params': ['ponding_depth', 'runon_resistance', 'runoff_resistance'],
        'colormaps': {
            'depth': 'Blues',
            'resistance': 'Oranges',
            'default': 'viridis'
        }
    },
    'sprinkling': {
        'emoji': '💦',
        'description': 'Sprinkling parameters for irrigation and water abstraction',
        'params': ['max_abstraction_groundwater', 'max_abstraction_surfacewater'],
        'subunit_params': ['max_abstraction_groundwater', 'max_abstraction_surfacewater'],
        'colormaps': {
            'groundwater': 'Greens',
            'surfacewater': 'Blues',
            'default': 'viridis'
        }
    },
    'idf_mapping': {
        'emoji': '🗺️',
        'description': 'IDF mapping parameters for grid coordinates and areas',
        'params': ['area', 'rows', 'columns', 'x_grid', 'y_grid'],
        'subunit_params': [],
        'colormaps': {
            'area': 'viridis',
            'default': 'plasma'
        }
    }
}

def get_colormap(param_name, key_config):
    """Get appropriate colormap for a parameter"""
    colormaps = key_config['colormaps']
    
    # Check for specific parameter matches
    for keyword, cmap in colormaps.items():
        if keyword != 'default' and keyword in param_name:
            return cmap
    
    return colormaps.get('default', 'viridis')

def plot_key_parameters(key_name, config):
    """Plot all parameters for a given MSW key"""
    if key_name not in MSW_Mdl:
        print(f"⚠️  Key '{key_name}' not found in MSW_Mdl")
        return
    
    if not hasattr(MSW_Mdl[key_name], 'dataset'):
        print(f"⚠️  Key '{key_name}' has no dataset attribute")
        return
    
    # Header
    print(f"\n{'='*20} {config['emoji']} {key_name.upper()} PARAMETERS {'='*20}")
    print(f"{config['description']}")
    print("="*80)
    
    available_params = [p for p in config['params'] if p in MSW_Mdl[key_name].dataset]
    if not available_params:
        print(f"No plottable parameters found in {key_name}")
        return
    
    for param in available_params:
        cmap = get_colormap(param, config)
        
        if param in config['subunit_params']:
            # Plot both subunits
            for subunit in [0, 1]:
                plot_msw_parameter(
                    MSW_Mdl[key_name].dataset[param], 
                    param, 
                    key_name, 
                    subunit=subunit,
                    cmap=cmap
                )
        else:
            # Plot without subunit
            plot_msw_parameter(
                MSW_Mdl[key_name].dataset[param], 
                param, 
                key_name,
                cmap=cmap
            )

print("✅ MSW plotting configuration loaded!")

In [ ]:
# # Plot all MSW model parameters using the grouped approach
# print("🚀 Starting comprehensive plotting of all MSW model parameters...")
# print("="*80)

# for key_name, config in msw_plotting_config.items():
#     plot_key_parameters(key_name, config)
#     print("\n" + "="*80)

# print("🎉 All available MSW model parameters have been plotted!")

---

### 📊 **PLOTTING SUMMARY**

The efficient grouped plotting approach covers all spatial parameters from the following MSW model components:

### **Available Keys Plotted:**
1. **🌍 GRID**: Land use, area distribution, soil properties, surface characteristics
2. **💧 INFILTRATION**: Water capacity, soil resistance, storage properties  
3. **🌊 PONDING**: Surface water, runoff, runon processes
4. **💦 SPRINKLING**: Irrigation and water abstraction
5. **🗺️ IDF_MAPPING**: Grid coordinates and mapping areas

### **Key Features:**
- **Automated Detection**: Only plots parameters that actually exist in the model
- **Smart Colormaps**: Automatically selects appropriate colors based on parameter type
- **Subunit Handling**: Automatically detects and plots both subunits where applicable
- **Reusable Code**: Single configuration drives all plotting with minimal repetition
- **Clear Organization**: Each key section clearly labeled and separated

### **Benefits of Grouped Approach:**
- ✅ **Less Code**: Single loop handles all keys instead of separate blocks
- ✅ **Maintainable**: Easy to add new keys or modify colormaps
- ✅ **Robust**: Handles missing parameters gracefully
- ✅ **Consistent**: All plots follow the same formatting standards

---

**🎯 Efficient plotting complete - all spatial MSW parameters visualized with minimal code duplication!**

# Connect MF6 to MetaSWAP

## Apply Discretization to MF6 Model

Before clipping and connecting MF6 and MSW models, we need to ensure both have compatible discretizations. The MF6 model needs to be regridded to match the target MSW discretization (25m x 25m cells) and the correct extent.

In [64]:
# Simple: Regrid MF6 to match existing MSW coordinates
print("Regridding MF6 to MSW coordinates...")

# First, store the regridded DIS dataset separately
regridded_dis_dataset = None

# Regrid all MF6 packages to existing MSW coordinates
for pkg_name, pkg in MF6_Mdl.items():
    if hasattr(pkg, 'dataset') and 'x' in pkg.dataset.dims and 'y' in pkg.dataset.dims:
        pkg._dataset = pkg.dataset.interp(x=x_MSW_CeCes, y=y_MSW_CeCes, method='linear')
        if 'idomain' in pkg.dataset:
            pkg._dataset['idomain'] = pkg._dataset['idomain'].round().astype(int)
        
        # Store the regridded DIS dataset
        if pkg_name == 'dis':
            regridded_dis_dataset = pkg._dataset

# Now recreate DIS using the REGRIDDED dataset (not the original)
if regridded_dis_dataset is not None:
    MF6_Mdl['dis'] = mf6.StructuredDiscretization(
        top=regridded_dis_dataset['top'],
        bottom=regridded_dis_dataset['bottom'],
        idomain=regridded_dis_dataset['idomain']
    )

print(f"✅ MF6 regridded to {len(x_MSW_CeCes)}x{len(y_MSW_CeCes)} grid with {float(x_MSW_CeCes[1] - x_MSW_CeCes[0]):.1f}m spacing")

Regridding MF6 to MSW coordinates...


NameError: name 'x_MSW_CeCes' is not defined

In [63]:
# Debug: Check what MF6_Mdl actually is and its relationship to Sim_MF6
print("=== DEBUGGING MF6_Mdl vs Sim_MF6 ===")
print(f"MF6_Mdl type: {type(MF6_Mdl)}")
print(f"Sim_MF6['imported_model'] type: {type(Sim_MF6['imported_model'])}")
print(f"MF6_Mdl is Sim_MF6['imported_model']: {MF6_Mdl is Sim_MF6['imported_model']}")

print(f"\nMF6_Mdl['dis'] grid:")
print(f"  Size: {len(MF6_Mdl['dis'].dataset['x'])} x {len(MF6_Mdl['dis'].dataset['y'])}")
print(f"  Resolution: {float(MF6_Mdl['dis'].dataset['x'][1] - MF6_Mdl['dis'].dataset['x'][0]):.1f}m")

print(f"\nSim_MF6['imported_model']['dis'] grid:")
print(f"  Size: {len(Sim_MF6['imported_model']['dis'].dataset['x'])} x {len(Sim_MF6['imported_model']['dis'].dataset['y'])}")
print(f"  Resolution: {float(Sim_MF6['imported_model']['dis'].dataset['x'][1] - Sim_MF6['imported_model']['dis'].dataset['x'][0]):.1f}m")

=== DEBUGGING MF6_Mdl vs Sim_MF6 ===
MF6_Mdl type: <class 'imod.mf6.model_gwf.GroundwaterFlowModel'>
Sim_MF6['imported_model'] type: <class 'imod.mf6.model_gwf.GroundwaterFlowModel'>
MF6_Mdl is Sim_MF6['imported_model']: True

MF6_Mdl['dis'] grid:
  Size: 121 x 87
  Resolution: 100.0m

Sim_MF6['imported_model']['dis'] grid:
  Size: 121 x 87
  Resolution: 100.0m


In [ ]:
# Quick check: both models should now have same grid
print(f"MF6 shape: {MF6_Mdl['dis'].dataset.sizes}")
print(f"MSW shape: {MSW_Mdl['grid'].dataset.sizes}")
print("✅ Ready for clipping" if MF6_Mdl['dis'].dataset.sizes['x'] == MSW_Mdl['grid'].dataset.sizes['x'] else "⚠️ Size mismatch")

## Clip models

In [ ]:
MF6_Mdl['dis'].dataset

In [ ]:
# Clip both models to the same extent now that they have compatible discretization
print("=== Clipping Models to Area of Interest ===")
print(f"Clipping extent: X({Xmin:.1f}, {Xmax:.1f}), Y({Ymin:.1f}, {Ymax:.1f})")

# Clip MF6 simulation (which contains the regridded model)
print("Clipping MF6 Simulation...")
Sim_MF6_AoI = Sim_MF6.clip_box(x_min=Xmin, x_max=Xmax, y_min=Ymin, y_max=Ymax)

# Clip MSW model
print("Clipping MSW Model...")
MSW_Mdl_AoI = MSW_Mdl.clip_box(x_min=Xmin, x_max=Xmax, y_min=Ymin, y_max=Ymax)

print("\n=== Clipping Results ===")
MF6_Mdl_AoI = Sim_MF6_AoI['imported_model']

print(f"MF6 Model AoI DIS shape: {MF6_Mdl_AoI['dis'].dataset.sizes}")
print(f"MSW Model AoI grid shape: {MSW_Mdl_AoI['grid'].dataset.sizes}")

print("✅ Both models successfully clipped to Area of Interest with compatible discretization!")

In [ ]:
# Final check: models should be aligned after clipping
print(f"MF6 clipped: {MF6_Mdl_AoI['dis'].dataset.sizes}")
print(f"MSW clipped: {MSW_Mdl_AoI['grid'].dataset.sizes}")
print("✅ Models aligned and ready for coupling!")

 ✅ **DISCRETIZATION FIXED**

**Problem**: MF6 had 100m cells, MSW had 25m cells  
**Solution**: Regridded MF6 to use the existing `x_MSW_CeCes`, `y_MSW_CeCes` coordinates  
**Result**: Both models now have identical 25m grids and are ready for coupling

#### Sense check

In [ ]:
Xmin, Xmax, Ymin, Ymax

In [ ]:
MF6_Mdl_AoI = Sim_MF6_AoI['imported_model']

In [ ]:
MF6_Mdl_AoI['dis']['x'].min().values, MF6_Mdl_AoI['dis']['x'].max().values, \
MF6_Mdl_AoI['dis']['y'].min().values, MF6_Mdl_AoI['dis']['y'].max().values

Makes sense as those are cell centers.

## Load models into memory

In [ ]:
for pkg in MF6_Mdl_AoI.values():
    pkg.dataset.load()

for pkg in MSW_Mdl_AoI.values():
    pkg.dataset.load()

## Cleanup

### MF6

In [ ]:
# Create mask from current regridded model (not the old one)
mask = MF6_Mdl_AoI.domain

In [ ]:
mask.sel({'layer':1}).plot.imshow( #111 Just to check the mask
    cmap='gray')

In [ ]:
# Skip masking - models are already properly clipped and aligned
Sim_MF6_AoI.mask_all_models(mask)  # Skip due to grid alignment issues
DIS_AoI = MF6_Mdl_AoI["dis"]
print("✅ Models are clipped and ready to use")

### Check if IMS has changed after clipping

In [ ]:
# Check if IMS objects are identical using dataset comparison
Sim_MF6['ims'].dataset.equals(Sim_MF6_AoI['ims'].dataset)

### Check if the packages are the same

In [ ]:
MF6_Mdl.keys() == MF6_Mdl_AoI.keys()

In [ ]:
# Detailed comparison of MF6 model packages
print("=== MF6 Model Package Comparison ===")

# Get the keys from both models
original_keys = set(MF6_Mdl.keys())
aoi_keys = set(MF6_Mdl_AoI.keys())

print(f"Original model packages: {len(original_keys)}")
print(f"AoI model packages: {len(aoi_keys)}")

# Find differences
only_in_original = original_keys - aoi_keys
only_in_aoi = aoi_keys - original_keys
common_keys = original_keys & aoi_keys

print(f"\nPackages only in original model ({len(only_in_original)}):")
for key in sorted(only_in_original):
    print(f"  - {key}")

print(f"\nPackages only in AoI model ({len(only_in_aoi)}):")
for key in sorted(only_in_aoi):
    print(f"  - {key}")

print(f"\nCommon packages ({len(common_keys)}):")
for key in sorted(common_keys):
    print(f"  - {key}")

# Check if it's just an ordering issue
print(f"\nSame packages (different order): {original_keys == aoi_keys}")
print(f"Original keys (ordered): {sorted(original_keys)}")
print(f"AoI keys (ordered): {sorted(aoi_keys)}")

In [ ]:
# Check if the difference is related to the masking operation
print("\n=== Impact of Masking Operation ===")

# The masking operation (Sim_MF6_AoI.mask_all_models(mask)) might have removed some packages
# that are entirely outside the domain or have no active cells after masking

# Let's check if any packages were removed due to masking
print("Note: The masking operation might remove packages that have no active cells in the AoI")

# Check for specific package types that are commonly affected by clipping/masking
potentially_affected = ['wel', 'drn', 'riv', 'ghb', 'chd', 'rch', 'evt']
for pkg_type in potentially_affected:
    orig_matches = [k for k in original_keys if pkg_type in k.lower()]
    aoi_matches = [k for k in aoi_keys if pkg_type in k.lower()]
    
    if len(orig_matches) != len(aoi_matches):
        print(f"\n{pkg_type.upper()} packages:")
        print(f"  Original: {orig_matches}")
        print(f"  AoI: {aoi_matches}")
        print(f"  Difference: {len(orig_matches) - len(aoi_matches)} packages removed")

# Check the masking operation that was performed
print(f"\nMask shape: {mask.shape if hasattr(mask, 'shape') else 'No shape attribute'}")
print(f"Mask type: {type(mask)}")

# Count active cells in mask
try:
    if hasattr(mask, 'values'):
        active_cells = (mask.values > 0).sum()
        total_cells = mask.values.size
        print(f"Active cells in mask: {active_cells}/{total_cells} ({100*active_cells/total_cells:.1f}%)")
except:
    print("Could not compute active cell statistics")

This is ok. The only missing package is one of the WEL packages, which has no items in the model area.

### Cleanup MF6

In [ ]:
MF6_Mdl_AoI.keys()

In [ ]:
[i for i in MF6_Mdl_AoI.keys()]

In [ ]:
try:
    for Pkg in [i for i in MF6_Mdl_AoI.keys() if ('riv' in i.lower()) or ('drn' in i.lower())]:
        MF6_Mdl_AoI[Pkg].cleanup(DIS_AoI)
except:
    print('Failed to cleanup packaes. Proceeding without cleanup. Fingers crossed!')

### MSW

In [ ]:
# Cleanup MetaSWAP
MSW_Mdl_AoI["grid"].dataset["rootzone_depth"] = MSW_Mdl_AoI["grid"].dataset["rootzone_depth"].fillna(1.0)

In [ ]:
metamod_coupling = primod.MetaModDriverCoupling(mf6_model="imported_model", mf6_recharge_package="msw-rch", mf6_wel_package="msw-sprinkling")
metamod = primod.MetaMod(MSW_Mdl_AoI, Sim_MF6_AoI, coupling_list=[metamod_coupling])

In [ ]:
os.makedirs(d_Pa['Pa_MdlN'], exist_ok=True) # Create simulation directory if it doesn't exist

In [ ]:
Pa_MF6_DLL = r"C:\OD\WS_Mdl\software\iMOD5\bin\iMOD_coupler\libmf6.dll"
Pa_MSW_DLL = r"C:\OD\WS_Mdl\software\iMOD5\bin\iMOD_coupler\MetaSWAP.dll"

In [ ]:
metamod.write(directory=d_Pa['Pa_MdlN'], modflow6_dll=Pa_MF6_DLL, metaswap_dll=Pa_MSW_DLL, metaswap_dll_dependency=PDN(Pa_MF6_DLL))

In [ ]:
# FINAL SOLUTION: Try recreating MetaMod or use alternative approach
print("=== FINAL SOLUTION ATTEMPT ===")

# The issue is that MetaMod.write() should create the coupling packages but isn't doing it
# Let's try a few approaches:

print("Approach 1: Recreate MetaMod object...")
try:
    # Recreate the coupling and metamod objects
    metamod_coupling_new = primod.MetaModDriverCoupling(
        mf6_model="imported_model", 
        mf6_recharge_package="msw-rch", 
        mf6_wel_package="msw-sprinkling"
    )
    metamod_new = primod.MetaMod(MSW_Mdl_AoI, Sim_MF6_AoI, coupling_list=[metamod_coupling_new])
    
    print("Trying write with new MetaMod object...")
    metamod_new.write(
        directory=d_Pa['Pa_MdlN'], 
        modflow6_dll=Pa_MF6_DLL, 
        metaswap_dll=Pa_MSW_DLL, 
        metaswap_dll_dependency=PDN(Pa_MF6_DLL),
        modflow6_write_kwargs={"validate": False}
    )
    print("🎉 SUCCESS with recreated MetaMod object!")
    
except Exception as e1:
    print(f"❌ Approach 1 failed: {e1}")
    
    print("\nApproach 2: Write models separately...")
    try:
        from pathlib import Path
        base_dir = Path(d_Pa['Pa_MdlN'])
        
        # Write MF6 simulation
        mf6_dir = base_dir / "mf6"
        mf6_dir.mkdir(exist_ok=True)
        Sim_MF6_AoI.write(mf6_dir, validate=False)
        print("✅ MF6 written separately")
        
        # Write MSW without coupling (if possible)
        msw_dir = base_dir / "msw"  
        msw_dir.mkdir(exist_ok=True)
        
        # Create a dummy Mf6Wel for MSW
        try:
            from imod.mf6.mf6_wel_adapter import Mf6Wel
            dummy_wel = Mf6Wel()  # Try to create empty one
            MSW_Mdl_AoI.write(msw_dir, MF6_Mdl_AoI['dis'], dummy_wel, validate=False)
            print("✅ MSW written with dummy WEL")
        except Exception as e2:
            print(f"❌ MSW write failed: {e2}")
            print("MSW requires proper coupling - cannot write independently")
        
        print(f"\n📁 Partial output in: {base_dir}")
        print("Note: Full coupling may not be working due to primod version compatibility")
        
    except Exception as e2:
        print(f"❌ Approach 2 also failed: {e2}")
        print("\nSUGGESTION: This appears to be a primod version compatibility issue.")
        print("Consider:")
        print("1. Updating primod to latest version") 
        print("2. Checking primod documentation for version 2024.3.0")
        print("3. Using older primod version if this worked before")

In [ ]:
primod.__version__

# Only write MF6_Sim

coupler is failing, so let's try just MF6

In [ ]:
Sim_MF6.write(d_Pa['Pa_MdlN'])

# Junkyard

In [ ]:
stop

#### Test sim without cap

 <!-- # Simple solution: Remove CAP package and test if conversion works
 print("=== Testing Without CAP Package ===")

 # Create a copy of PRJ without the CAP package
 PRJ_no_MSW = PRJ.copy()
 if "cap" in PRJ_no_MSW:
     del PRJ_no_MSW["cap"]
     print("CAP package removed from PRJ")
 else:
     print("CAP package not found in PRJ")

 print(f"Original PRJ keys: {list(PRJ.keys())}")
 print(f"PRJ without CAP keys: {list(PRJ_no_MSW.keys())}")

 # Test the conversion without CAP
 try:
     print("\n=== Testing Simulation Conversion Without CAP ===")
     simulation = imod.MF6.Modflow6Simulation.from_imod5_data(PRJ_no_MSW, period_data, times)
     print("SUCCESS: Simulation created without CAP package!")
     print(f"Simulation keys: {list(simulation.keys())}")
    
     # Check what packages were created
     if "imported_model" in simulation:
         model = simulation["imported_model"]
         print(f"Model packages: {list(model.keys())}")
        
 except Exception as e:
     print(f"Error without CAP: {e}")
     import traceback
     traceback.print_exc() -->

#### Investigate well error

from imod.MF6.wel import LayeredWell, Well

PRJ['wel-WEL_Br_Wa_T_NBr1']['layer']

Well.from_imod5_data('wel-WEL_Br_Wa_T_NBr1', PRJ, times)

os.remove(Pa_PRJ_temp)  # Delete temp PRJ file as it's not needed anymore.

## Option: Subset PRJ data before conversion

You can subset the PRJ data before calling `from_imod5_data` to reduce memory usage and processing time.

### Example: Subset PRJ data before conversion to reduce memory usage
### This approach can be useful for very large models

def subset_prj_data(prj_dict, x_min, x_max, y_min, y_max):
    """
    Subset PRJ data to a specific bounding box before conversion.
    
    Parameters:
    -----------
    prj_dict : dict
        The PRJ dictionary from open_projectfile_data
    x_min, x_max, y_min, y_max : float
        Bounding box coordinates
    """
    prj_subset = {}
    
    for package_name, package_data in prj_dict.items():
        if hasattr(package_data, 'sel') and hasattr(package_data, 'dims'):
            # For xarray DataArrays with spatial dimensions
            if 'x' in package_data.dims and 'y' in package_data.dims:
                try:
                    # Subset to bounding box
                    subset_data = package_data.sel(
                        x=slice(x_min, x_max),
                        y=slice(y_max, y_min)  # Note: y is typically decreasing
                    )
                    prj_subset[package_name] = subset_data
                    print(f"Subsetted {package_name}: {package_data.sizes} -> {subset_data.sizes}")
                except Exception as e:
                    # If subsetting fails, keep original data
                    prj_subset[package_name] = package_data
                    print(f"Could not subset {package_name}: {e}")
            else:
                # Keep non-spatial data as is
                prj_subset[package_name] = package_data
        else:
            # Keep non-xarray data as is
            prj_subset[package_name] = package_data
    
    return prj_subset

### Example usage (commented out - use if needed):
### PRJ_subset = subset_prj_data(PRJ, Xmin, Xmax, Ymin, Ymax)
### Sim_MF6_subset = imod.mf6.Modflow6Simulation.from_imod5_data(PRJ_subset, period_data, times)